In [1]:
# Import Dependencies
import pandas as pd
from splinter import Browser
from bs4 import BeautifulSoup
import time

In [2]:
# Create browser to navigate oracle
executable_path = {'executable_path': 'chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=True)
url = 'https://gatherer.wizards.com/Pages/Default.aspx'
browser.visit(url)

In [3]:
# Function to remove formatting from gatherer
def extract_name(card, card_long):
    length = len(card_long)
    good_string = card_long[length-len(card):-1]
    good_string += card_long[length-1]
    return(good_string) 

In [4]:
# Function to scrape oracle text, then relevant data is crafted into dataframe and a list of mismatched links, then returned.
def scrape_oracle(cards_data):

    # Empty list for list of mismatched links is crafted
    bad_link = []

    # Empty lists are initialized
    card_list = []
    gen_list = []
    snow_list = []
    c_list = []
    v_list = []
    two_b_list = []
    phy_list = []
    w_list = []
    u_list = []
    b_list = []
    r_list = []
    g_list = []
    cmc_list = []
    type_line_list = []
    text_list = []
    power_list = []
    toughness_list = []
    loyalty_list = []

    # For loop loops through list of cards
    for row in cards_data.iterrows():

        # Initialize all mana values at 0
        gen = 0
        snow = 0
        c = 0
        v = 0
        two_b = 0
        phy = 0
        w = 0
        u = 0
        b = 0
        r = 0
        g = 0

        # Navigates browser to card page. 
        # Check if oracle id has been manually included.  Otherwise, use search bar.
        try:
            url = "https://gatherer.wizards.com/Pages/Card/Details.aspx?multiverseid=" + str(row[1]["Oracle ID"])
            browser.visit(url)
            time.sleep(3)
            search = False

        #  Navigate to search bar. Note that this picks the first option in the dropdown when a card name is entered.
        # It is possible that the wrong card will be grabbed, so this is verified below.
        except:
            browser.fill("ctl00$ctl00$MainContent$Content$SearchControls$CardSearchBoxParent$CardSearchBox", row[1]["Name"])
            time.sleep(3)
            browser.links.find_by_partial_text(row[1]["Name"]).click()
            search = True

        # Soup object is crafted at card page so html can be read
        html = browser.html
        hot_soup = BeautifulSoup(html, "html.parser")


        ############# NAME ############# 


        # Check to see if card navigation was successful.
        # Attempt is made to find string containing name. This only fails if card is double-faced (HTML IDs change).
        # "try" and "except" do same thing except the list of HTML ID's differ.
        # In each, name must be extracted from formatting to check if strings are equal.
        try:
            card_long = browser.find_by_id("ctl00_ctl00_ctl00_MainContent_SubContent_SubContent_nameRow").text.split("\n")[1]
            card_name = extract_name(row[1]["Name"], card_long)
            html_ids = ["ctl00_ctl00_ctl00_MainContent_SubContent_SubContent_typeRow",
                        "ctl00_ctl00_ctl00_MainContent_SubContent_SubContent_textRow",
                        "ctl00_ctl00_ctl00_MainContent_SubContent_SubContent_ptRow"]
        except:
            card_long = browser.find_by_id("ctl00_ctl00_ctl00_MainContent_SubContent_SubContent_ctl02_nameRow")\
                .text.split("\n")[1]
            card_name = extract_name(row[1]["Name"], card_long)
            html_ids = ["ctl00_ctl00_ctl00_MainContent_SubContent_SubContent_ctl02_typeRow",
                "ctl00_ctl00_ctl00_MainContent_SubContent_SubContent_ctl02_textRow",
                "ctl00_ctl00_ctl00_MainContent_SubContent_SubContent_ctl02_ptRow"]
        
        # If strings are not equal, the entry is entered into a list, then skipped.  Otherwise, the program continues.
        if row[1]["Name"] != card_name:
                bad_link.append(row[1]["Name"])

        # Card name is correct,so data is scraped
        else:


            ############# MANA #############


            # Find the element where images for mana symbols would be if present.
            mana_row = hot_soup.find("td", class_="rightCol").find_all("div", class_="row")[1].find_all("img")
            
            # Check if list of images is empty, implying no mana cost.  If one does not exist, skip ahead.
            if mana_row != []:  
                
                # Loop through all mana symbols in the mana row, in each case the alt text will be used.
                # Mana is saved as [gen]eric, [snow], [c]olorless, [v]ariable, (two_b)rid, (phy)rexian, and
                #    [w]hite, bl[u]e, [b]lack, [r]ed, and [g]reen (hybrid, phyrexian, and 2-brid mana are counted twice).
                for cost in mana_row:

                    # Attempt to turn symbol into a number, only possible if it is generic
                    try:
                        gen = int(cost["alt"])

                    # Symbol is not generic mana, so it is an image with alt text to be checked.
                    # Once mana symbol is determined, it increments the relevant variable by 1.
                    except:

                        # These three qualities are unique, and thus mutually exclusive
                        if "Snow" == cost["alt"]:
                            snow += 1
                        elif "Colorless" == cost["alt"]:
                            c += 1
                        elif "Variable Colorless" == cost["alt"]:
                            v += 1

                        # The following mana types may appear in combination in the same symbol, thus are checked if string
                        #    is in the alt text. The intent is that mana which may be paid for in multiple ways is counted
                        #    as such.  For example, "Phyrexian Red" will increment both "phy" and "r"
                        else:
                            if "Two or " in cost["alt"]:
                                two_b += 1
                            if "Phyrexian" in cost["alt"]:
                                phy += 1
                            if "White" in cost["alt"]:
                                w += 1
                            if "Blue" in cost["alt"]:
                                u += 1
                            if "Black" in cost["alt"]:
                                b += 1
                            if "Red" in cost["alt"]:
                                r += 1
                            if "Green" in cost["alt"]:
                                g += 1    


            # If no mana cost, return all mana entries as None ("None" is different from "0", see Mountain and Ornithopter)
            else:
                gen = None
                snow = None
                c = None
                v = None
                two_b = None
                phy = None
                w = None
                u = None
                b = None
                r = None
                g = None


            ############# CONVERTED MANA COST #############


            # Scrape a converted mana cost. If one does not exist, mark cmc as None
            # In order to ensure that the cmc is a number and usable, it must be altered from that on oracle.
            # Here, we abuse the fact that the highest cmc in pauper is 9, so all numbers will be 1 character.
            # Number is then extracted from Wizard's formatting.
            try:
                ugly_number = hot_soup.find("td", class_="rightCol").find_all("div", class_="row")[2].\
                    find("div", class_="value").text
                cmc = int(ugly_number[len(ugly_number) - 1])
            except:
                cmc = None 



            ############# TYPE-LINE #############


            # Scrape the type-line.  Every card has one, without exception.
            type_line = browser.find_by_id(html_ids[0]).text.split("\n")[1]


            ############# TEXT BOX #############


            # Check for a text box, scrape if one is present, then reformat. If one does not exist, mark text as None
            try:
                text = browser.find_by_id(html_ids[1]).\
                    text.replace("Card Text:\n", "")
            except:
                text = None


            ############# POWER, TOUGHNESS, LOYALTY #############


            # Check for power and toughness. If both are valid, set loyalty to zero.
            try:
                pt = browser.find_by_id(html_ids[2]).text.split("\n")[1].\
                        split(" / ")
                power = int(pt[0])
                toughness = int(pt[1])
                loyalty = None
            # Something failed so card has has no pt. Next, check if card has loyalty (stored on same line).
            except:
                power = None
                toughness = None
                try:
                    loyalty = int(browser.find_by_id(html_ids[2]).text.\
                                  split("\n")[1])
                except:
                    loyalty = None

        # Appends the card data to the relevant lists
        card_list.append(row[1]["Name"])
        gen_list.append(gen)
        snow_list.append(snow)
        c_list.append(c)
        v_list.append(v)
        two_b_list.append(two_b)
        phy_list.append(phy)
        w_list.append(w)
        u_list.append(u)
        b_list.append(b)
        r_list.append(r)
        g_list.append(g)
        cmc_list.append(cmc)
        type_line_list.append(type_line)
        text_list.append(text)
        power_list.append(power)
        toughness_list.append(toughness)
        loyalty_list.append(loyalty)


        # Navigates back to search page, if necessary
        if search:
            browser.back()

    # Dataframe is crafted, then returned.
    scraped_df= pd.DataFrame({"Name": card_list,
                              "Cost Generic": gen_list,
                              "Cost Snow": snow_list,
                              "Cost Colorless": c_list,
                              "Cost Variable": v_list,
                              "Cost 2-brid": two_b_list,
                              "Cost Phyrexian": phy_list,
                              "Cost White": w_list,
                              "Cost Blue": u_list,
                              "Cost Black": b_list,
                              "Cost Red": r_list,
                              "Cost Green": g_list,
                              "Converted Mana Cost": cmc_list,
                              "Type Line": type_line_list,
                              "Text Box": text_list,
                              "Power": power_list,
                              "Toughness": toughness_list,
                              "Loyalty": loyalty_list
                          })
    # Return finished dataframe and list of mismatched links
    return(scraped_df, bad_link)

In [5]:
# Card data is read from csv and then run through the scrape oracle function, returning the dataframe and a list of names 
#    which did not link to the correct page
cards_data = pd.read_csv("raw_count.csv")
cards_df, bad_link = scrape_oracle(cards_data)
cards_df

,Name,Cost Generic,Cost Snow,Cost Colorless,Cost Variable,Cost 2-brid,Cost Phyrexian,Cost White,Cost Blue,Cost Black,Cost Red,Cost Green,Converted Mana Cost,Type Line,Text Box,Power,Toughness,Loyalty
0,Ghitu Lavarunner,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,Creature — Human Wizard,As long as there are two or more instant and/o...,1.0,2.0,None
1,Thermo-Alchemist,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,Creature — Human Shaman,Defender\n: Thermo-Alchemist deals 1 damage to...,0.0,3.0,None
2,Chain Lightning,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,Sorcery,Chain Lightning deals 3 damage to any target. ...,NaN,NaN,None
3,Lava Spike,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,Sorcery — Arcane,Lava Spike deals 3 damage to target player or ...,NaN,NaN,None
4,Lightning Bolt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,Instant,Lightning Bolt deals 3 damage to any target.,NaN,NaN,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
139,Urban Utopia,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,Enchantment — Aura,Enchant land\nWhen Urban Utopia enters the bat...,NaN,NaN,None
140,Snow-Covered Forest,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Basic Snow Land — Forest,G,NaN,NaN,None
141,Snow-Covered Plains,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Basic Snow Land — Plains,W,NaN,NaN,None
142,Duress,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,Sorcery,Target opponent reveals their hand. You choose...,NaN,NaN,None


In [6]:
# Turns list of bad links into a dataframe for later merging
bad_df = pd.DataFrame({"Name": bad_link})
bad_df

,Name
0,Mountain
1,Snap
2,Island
3,Forest
4,Atog


In [7]:
# Drop all cards whose link did not go to the correct card
cards_df.drop(cards_df.loc[cards_df["Name"].isin(bad_link)].index, inplace=True)

In [8]:
# A dataframe of cards known to cause issues when the above process is run is manually entered then merged with the bad_df.
bad_ids = pd.DataFrame({"Name": ["Island", "Swamp", "Mountain", "Forest", "Ponder", "Snap", "Dispel", "Atog"],
                       "Oracle ID": [491574, 491576, 491578, 491580, 451051, 426582, 401858, 202463]})
card_id_df = bad_df.merge(bad_ids, how="inner", on="Name")

# The dataframe is then sent through the scrape oracle function.  Bad link is 
card_manual_df, bad_link = scrape_oracle(card_id_df)

if bad_link != []:
    print(bad_link)
    print("The above list contains cards whose link did not work.  Please change the id to ensure they function.")

In [9]:
# Dataframes of cards which did and did not navigate are appended to form one another to generate a complete dataframe
card_final_df = cards_df.append(card_manual_df).reset_index(drop=True)
card_final_df.head(50)

,Name,Cost Generic,Cost Snow,Cost Colorless,Cost Variable,Cost 2-brid,Cost Phyrexian,Cost White,Cost Blue,Cost Black,Cost Red,Cost Green,Converted Mana Cost,Type Line,Text Box,Power,Toughness,Loyalty
0,Ghitu Lavarunner,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,Creature — Human Wizard,As long as there are two or more instant and/o...,1.0,2.0,None
1,Thermo-Alchemist,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,2.0,Creature — Human Shaman,Defender\n: Thermo-Alchemist deals 1 damage to...,0.0,3.0,None
2,Chain Lightning,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,Sorcery,Chain Lightning deals 3 damage to any target. ...,NaN,NaN,None
3,Lava Spike,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,Sorcery — Arcane,Lava Spike deals 3 damage to target player or ...,NaN,NaN,None
4,Lightning Bolt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,Instant,Lightning Bolt deals 3 damage to any target.,NaN,NaN,None
5,Needle Drop,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,Instant,Needle Drop deals 1 damage to any target that ...,NaN,NaN,None
6,Searing Blaze,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,2.0,Instant,Searing Blaze deals 1 damage to target player ...,NaN,NaN,None
7,Rift Bolt,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,3.0,Sorcery,Rift Bolt deals 3 damage to any target.\nSuspe...,NaN,NaN,None
8,Skewer the Critics,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,3.0,Sorcery,Spectacle (You may cast this spell for its spe...,NaN,NaN,None
9,Fireblast,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,6.0,Instant,You may sacrifice two Mountains rather than pa...,NaN,NaN,None


In [10]:
# Dataframe is output to csv
card_final_df.to_csv("mtg_merged_df.csv")